In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Cài nnU-Net bản chuẩn từ PyPI (Nhanh và sạch)
!pip install nnunetv2

# Cài thêm các thư viện bổ trợ nếu thiếu
!pip install nibabel matplotlib medpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 8.0 MB/s eta 0:00:00
  Created wheel for nnunetv2: filename=nnunetv2-2.6.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=cadc7adccbd257a0a7d4c7b4e97a25d73f2c6ecfbe854b202eb985a3ec1f2db2
  Stored in directory: /root/.cache/pip/wheels/89/5a/f8/b3def53b9c2133d2f8698ea2173bb5df63bd8e761ce8e9aec9
Successfully built medpy


## 1. Giải nén foler input

In [1]:
!ls

drive  sample_data


### a. Giải nén `nnUNet_preprocessed` vào thư mục content của Colab

In [2]:
import os
import zipfile
from tqdm import tqdm
import shutil

In [3]:
# GIẢI NÉN DỮ LIỆU PREPROCESSED
# ----------------------------------------------------------------------

os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
os.environ['nnUNet_preprocessed'] = "/content/nnUNet_preprocessed"
os.environ['nnUNet_results'] = "/content/drive/MyDrive/XUM_project/nnUNet_results"

!mkdir -p $nnUNet_raw
!mkdir -p $nnUNet_preprocessed

zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_preprocessed.zip"
zip_local = "/content/nnUNET_preprocessed.zip"
extract_root = "/content/nnUNet_preprocessed"

print("📦 Copy file từ Drive…")
!cp "$zip_path_drive" "$zip_local"

print("💨 Copy xong. Bắt đầu giải nén…")

with zipfile.ZipFile(zip_local, 'r') as zip_ref:
    # Detect folder root name inside zip (vd: nnUNET_preprocessed/)
    root_name = zip_ref.namelist()[0].split('/')[0]

    file_list = zip_ref.infolist()
    total_files = len(file_list)

    print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
    print("🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…")

    for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping"):
        file_path = file.filename

        # Bỏ lớp folder đầu tiên
        fixed_path = file_path.replace(root_name + "/", "", 1)

        if fixed_path.strip() == "":
            continue

        # Chỉ định nơi lưu đúng
        target_path = os.path.join(extract_root, fixed_path)

        # Tạo thư mục nếu cần
        if file.is_dir():
            os.makedirs(target_path, exist_ok=True)
        else:
            os.makedirs(os.path.dirname(target_path), exist_ok=True)
            with zip_ref.open(file) as source, open(target_path, "wb") as target:
                target.write(source.read())

os.remove(zip_local)

print("🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥")

📦 Copy file từ Drive…
💨 Copy xong. Bắt đầu giải nén…
🔧 Phát hiện folder root trong ZIP: nnUNET_preprocessed
🔧 Đang giải nén, loại bỏ lớp folder đầu tiên…


🔄 Unzipping: 100%|██████████| 2585/2585 [03:23<00:00, 12.72it/s]


🎉 GIẢI NÉN XONG VÀ ĐÃ TỰ ĐỘNG FIX FOLDER DOUBLE-LAYER! 😎🔥


### b. Giải nén `nnUNet_raw` vào thư mục content của Colab

In [4]:
# GIẢI NÉN DỮ LIỆU RAW (QUAN TRỌNG CHO INFERENCE)
# ----------------------------------------------------------------------

# 1. Thiết lập đường dẫn (Đảm bảo biến môi trường vẫn đúng)
os.environ['nnUNet_raw'] = "/content/nnUNet_raw"
!mkdir -p $nnUNet_raw

# Đường dẫn file zip trên Drive (Bạn kiểm tra lại tên file cho đúng nhé)
# Giả sử bạn lưu là nnUNet_raw.zip trong thư mục XUM_project
zip_path_drive = "/content/drive/MyDrive/XUM_project/nnUNET_raw.zip"
zip_local = "/content/nnUNet_raw.zip"
extract_root = "/content/nnUNet_raw"

if not os.path.exists(zip_path_drive):
    print(f"❌ Lỗi: Không tìm thấy file {zip_path_drive}")
    print("👉 Hãy kiểm tra lại tên file zip raw trên Google Drive của bạn.")
else:
    print("📦 Copy file Raw Data từ Drive...")
    !cp "$zip_path_drive" "$zip_local"

    print("💨 Copy xong. Bắt đầu giải nén Raw Data...")

    with zipfile.ZipFile(zip_local, 'r') as zip_ref:
        # Detect folder root name inside zip (ví dụ: nnUNet_raw/)
        # Để tránh bị lồng thư mục kiểu /content/nnUNet_raw/nnUNet_raw/Dataset...
        root_name = zip_ref.namelist()[0].split('/')[0]

        file_list = zip_ref.infolist()
        total_files = len(file_list)

        print(f"🔧 Phát hiện folder root trong ZIP: {root_name}")
        print("🔧 Đang giải nén và loại bỏ lớp folder thừa...")

        for file in tqdm(file_list, total=total_files, desc="🔄 Unzipping Raw"):
            file_path = file.filename

            # Logic: Nếu file nằm trong folder root, ta bỏ tên root đi để file nhảy ra ngoài
            # Ví dụ: nnUNet_raw/Dataset101/abc.nii -> Dataset101/abc.nii
            if file_path.startswith(root_name + "/"):
                fixed_path = file_path.replace(root_name + "/", "", 1)
            else:
                fixed_path = file_path # Nếu nén không có folder root thì giữ nguyên

            if fixed_path.strip() == "":
                continue

            # Chỉ định nơi lưu đúng
            target_path = os.path.join(extract_root, fixed_path)

            # Tạo thư mục nếu cần
            if file.is_dir():
                os.makedirs(target_path, exist_ok=True)
            else:
                os.makedirs(os.path.dirname(target_path), exist_ok=True)
                with zip_ref.open(file) as source, open(target_path, "wb") as target:
                    target.write(source.read())

    os.remove(zip_local)
    print("🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀")

📦 Copy file Raw Data từ Drive...
💨 Copy xong. Bắt đầu giải nén Raw Data...
🔧 Phát hiện folder root trong ZIP: nnUNET_raw
🔧 Đang giải nén và loại bỏ lớp folder thừa...


🔄 Unzipping Raw: 100%|██████████| 1845/1845 [04:29<00:00,  6.84it/s]

🎉 ĐÃ GIẢI NÉN RAW DATA THÀNH CÔNG! SẴN SÀNG CHẠY INFERENCE! 🚀


## 2. QU BraTS 2020 evaluating

$$Score = \frac{AUC_{Dice} + (1 - AUC_{FTP}) + (1 - AUC_{FTN})}{3}$$

In [5]:
# Phân tích cho EDL (Mặc định)
!python /content/drive/MyDrive/XUM_project/src/analysis_ause.py --mode edl

# Test thử trên 5 ca đầu tiên cho nhanh
# !python /content/drive/MyDrive/XUM_project/src/analysis_ause.py --mode edl --limit 5

🚀 STARTING ANALYSIS (QU-BRATS STANDARD 0-100) | MODE: EDL
🔍 Found 74 cases.

📊 Class: WT
Processing: 100% 74/74 [05:25<00:00,  4.40s/it]

📊 Class: TC
Processing: 100% 74/74 [01:22<00:00,  1.12s/it]

📊 Class: ET
Processing: 100% 74/74 [01:22<00:00,  1.11s/it]

                        🏆 QU-BRATS METRICS REPORT (V13)                         
Class  QU_Score  AUC_Dice_Total  AUC_FTP  AUC_FTN
   WT    0.9145          0.9443   0.1663   0.0346
   TC    0.9077          0.9237   0.1497   0.0508
   ET    0.8919          0.8589   0.1275   0.0558

✅ Results saved to: /content/drive/MyDrive/XUM_project/inference_results_edl/analysis_qu_brats_v13
--------------------------------------------------------------------------------


# 3. Giả định chiến thuật team Scan (top 1 - task uncertainty)

1. Logic của Team SCAN
Quy tắc: Nếu mô hình dự đoán một voxel là Khối u (Positive), hãy gán độ bất định của voxel đó bằng 0 (Tuyệt đối tin tưởng).

2. Mục đích: Để đảm bảo FTP (Filtered True Positive) luôn bằng 0. Khi bạn không bao giờ lọc bỏ dự đoán dương tính, bạn sẽ không bao giờ bị trừ điểm FTP.

=> Nó không thực tế và gây nguy hiểm, nhưng là chiến thuật tối ưu hóa điểm

In [9]:
!python /content/drive/MyDrive/XUM_project/src/run_scan_experiment.py --mode edl


🧪 BẮT ĐẦU THÍ NGHIỆM: TEAM SCAN STRATEGY | MODE: EDL

📊 Simulating SCAN Strategy for: WT
Processing: 100% 74/74 [00:43<00:00,  1.70it/s]

📊 Simulating SCAN Strategy for: TC
Processing: 100% 74/74 [00:45<00:00,  1.63it/s]

📊 Simulating SCAN Strategy for: ET
Processing: 100% 74/74 [00:41<00:00,  1.80it/s]

             🧪 KẾT QUẢ THÍ NGHIỆM SCAN STRATEGY             
Class  SCAN_Score  AUC_Dice  AUC_FTP  AUC_FTN
   WT      0.9604    0.9408   0.0250   0.0346
   TC      0.9475    0.9182   0.0250   0.0508
   ET      0.9180    0.8348   0.0250   0.0558
------------------------------------------------------------
